<a href="https://colab.research.google.com/github/douniagh/MyApplication-/blob/main/DeepLearning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df.Attack_type.value_counts()

Normal                   10066373
DDoS_UDP                  3201626
DDoS_ICMP                 2914354
DDoS_TCP                  2020120
Password                  1053385
DDoS_HTTP                  229022
Vulnerability_scanner      145869
SQL_injection               51203
Backdoor                    49724
Uploading                   37634
Port_Scanning               22564
XSS                         15915
Ransomware                  10925
MITM                         2458
OS_Fingerprinting            2002
Name: Attack_type, dtype: int64

## **Data Transformation**

Convert tabular data to images Procedures:

Use quantile transform to transform the original data samples

1  Élément de liste

into the scale of [0,255], representing pixel values
Generate images for each category (Normal, DoS, Fuzzy, Gear, RPM), each image consists of 27 data samples with 9 features. Thus, the size of each image is 993, length 9, width 9, and 3 color channels (RGB).

In [ ]:
!pip install dask dask-ml

In [ ]:
!pip install numpy==1.24.3

In [ ]:

import pandas as pd

# Assuming you have your DataFrame 'df'

# List of columns to drop
columns_to_drop = ['mqtt.conack.flags', 'arp.opcode', 'udp.port', 'udp.time_delta',
                   'dns.qry.name.len', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission',
                   'dns.retransmit_request', 'dns.retransmit_request_in', 'icmp.unused',
                   'mqtt.conflag.cleansess', 'mqtt.conflags', 'icmp.transmit_timestamp',
                   'mqtt.msg_decoded_as', 'mqtt.msg', 'mqtt.proto_len', 'mqtt.protoname',
                   'mqtt.topic', 'mqtt.topic_len', 'mbtcp.trans_id', 'mbtcp.unit_id']

# Drop the specified columns
df = df.drop(columns=columns_to_drop)


In [ ]:
# Drop duplicates
df = df.drop_duplicates()

# Handle missing values
df = df.dropna()

In [ ]:
df.head()

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,http.file_data,http.content_length,...,tcp.srcport,udp.stream,dns.qry.name,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.ver,mbtcp.len,Attack_label,Attack_type
0,0,72611,32675,0,0,0,19187,7091,0,0,...,0,0,0,0,0,0,0,0,0,DDoS_ICMP
1,1,29380,32675,0,0,0,18931,7102,0,0,...,0,0,0,0,0,0,0,0,0,DDoS_ICMP
2,2,100682,32675,0,0,0,18675,7114,0,0,...,0,0,0,0,0,0,0,0,0,DDoS_ICMP
3,3,83941,32675,0,0,0,18419,7125,0,0,...,0,0,0,0,0,0,0,0,0,DDoS_ICMP
4,4,91591,32675,0,0,0,18163,7136,0,0,...,0,0,0,0,0,0,0,0,0,DDoS_ICMP


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Define a function to apply scaling to a partition
def scale_partition(partition):
    for col in partition.columns:
        # Skip the label column
        if col == 'Attack_type':
            continue
        # Apply scaling to non-numeric columns
        if partition[col].dtype == 'int64':  # Assuming the encoded data type is int64
            partition[col] = scaler.fit_transform(partition[col].values.reshape(-1, 1))
    return partition

# Apply scaling to each partition in the DataFrame
scaled_df = df.map_partitions(scale_partition)

# Compute the scaled DataFrame
scaled_df = scaled_df.compute()

# Print the scaled DataFrame
print(scaled_df.head())

In [ ]:

from sklearn.preprocessing import MinMaxScaler,QuantileTransformer

# Initialize the MinMaxScaler
scaler = QuantileTransformer()

# Define a function to apply scaling to a partition
def scale_partition(partition):
    for col in partition.columns:
        # Skip the label column
        if col == 'Attack_type':
            continue
        # Apply scaling to non-numeric columns
        if partition[col].dtype == 'int64':  # Assuming the encoded data type is int64
            partition[col] = scaler.fit_transform(partition[col].values.reshape(-1, 1))
    return partition

# Apply scaling to each partition in the DataFrame
scaled_df = df.map_partitions(scale_partition)

# Compute the scaled DataFrame
scaled_df = scaled_df.compute()

# Print the scaled DataFrame
print(scaled_df.head())

   frame.time  ip.src_host  ip.dst_host  arp.dst.proto_ipv4  arp.hw.size  \
0    0.000000     0.917372     0.954955                 0.0          0.0   
1    0.000334     0.806023     0.954955                 0.0          0.0   
2    0.000667     0.950675     0.954955                 0.0          0.0   
3    0.001502     0.936735     0.954955                 0.0          0.0   
4    0.004505     0.943513     0.954955                 0.0          0.0   

   arp.src.proto_ipv4  icmp.checksum  icmp.seq_le  http.file_data  \
0                 0.0       0.900135     0.862567             0.0   
1                 0.0       0.899654     0.862592             0.0   
2                 0.0       0.899165     0.862621             0.0   
3                 0.0       0.898385     0.862646             0.0   
4                 0.0       0.897691     0.862672             0.0   

   http.content_length  ...  tcp.srcport  udp.stream  dns.qry.name  \
0                  0.0  ...          0.0         0.0      

In [ ]:

# Save the encoded DataFrame to a CSV file
scaled_df.to_csv('/content/drive/MyDrive/dl_scaled.csv', index=False)

In [ ]:

# Save the scaled DataFrame to a CSV file
scaled_df.to_csv('/content/drive/MyDrive/dlscaled.csv', index=False)

In [ ]:
df.describe()

In [2]:
import pandas as pd

# Define the file path
file_path = '/content/drive/MyDrive/dl_scaled.csv'

df = pd.read_csv(file_path,low_memory= False)
df.head()

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,http.file_data,http.content_length,...,tcp.srcport,udp.stream,dns.qry.name,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.ver,mbtcp.len,Attack_label,Attack_type
0,0.000000,0.917372,0.954955,0.0,0.0,0.0,0.900135,0.862567,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_ICMP
1,0.000334,0.806023,0.954955,0.0,0.0,0.0,0.899654,0.862592,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_ICMP
2,0.000667,0.950675,0.954955,0.0,0.0,0.0,0.899165,0.862621,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_ICMP
3,0.001502,0.936735,0.954955,0.0,0.0,0.0,0.898385,0.862646,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_ICMP
4,0.004505,0.943513,0.954955,0.0,0.0,0.0,0.897691,0.862672,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_ICMP


## **Generate images for each class**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
df_normal = df[df['Attack_type'] == 'Normal'].drop(['Attack_type'], axis=1)
df_ddos_udp = df[df['Attack_type'] == 'DDoS_UDP'].drop(['Attack_type'], axis=1)
df_ddos_icmp = df[df['Attack_type'] == 'DDoS_ICMP'].drop(['Attack_type'], axis=1)
df_ddos_tcp = df[df['Attack_type'] == 'DDoS_TCP'].drop(['Attack_type'], axis=1)
df_password = df[df['Attack_type'] == 'Password'].drop(['Attack_type'], axis=1)
df_ddos_http = df[df['Attack_type'] == 'DDoS_HTTP'].drop(['Attack_type'], axis=1)
df_vulnerability_scanner = df[df['Attack_type'] == 'Vulnerability_scanner'].drop(['Attack_type'], axis=1)
df_sql_injection = df[df['Attack_type'] == 'SQL_injection'].drop(['Attack_type'], axis=1)
df_backdoor = df[df['Attack_type'] == 'Backdoor'].drop(['Attack_type'], axis=1)
df_uploading = df[df['Attack_type'] == 'Uploading'].drop(['Attack_type'], axis=1)
df_port_scanning = df[df['Attack_type'] == 'Port_Scanning'].drop(['Attack_type'], axis=1)
df_xss = df[df['Attack_type'] == 'XSS'].drop(['Attack_type'], axis=1)
df_ransomware = df[df['Attack_type'] == 'Ransomware'].drop(['Attack_type'], axis=1)
df_mitm = df[df['Attack_type'] == 'MITM'].drop(['Attack_type'], axis=1)
df_os_fingerprinting = df[df['Attack_type'] == 'OS_Fingerprinting'].drop(['Attack_type'], axis=1)

In [4]:
# Assuming you have the number of samples for each attack type in a dictionary named 'attack_samples'
attack_samples = {
    "Normal": 10066373,
    "DDoS_UDP": 3201626,
    "DDoS_ICMP": 2914354,
    "DDoS_TCP": 2020120,
    "Password": 1053385,
    "DDoS_HTTP": 229022,
    "Vulnerability_scanner": 145869,
    "SQL_injection": 51203,
    "Backdoor": 49724,
    "Uploading": 37634,
    "Port_Scanning": 22564,
    "XSS": 15915,
    "Ransomware": 10925,
    "MITM": 2458,
    "OS_Fingerprinting": 2002
}

# Calculate the approximate image size based on the number of features per sample
features_per_sample = 63

image_sizes = {}
for attack_type, num_samples in attack_samples.items():
    elements_per_image = min(num_samples * features_per_sample, features_per_sample)
    width = int(np.sqrt(elements_per_image))
    height = int(np.ceil(elements_per_image / width))
    channels = 3  # RGB images
    image_sizes[attack_type] = (height, width, channels)

print("Image Sizes for Each Attack Type:")
print(image_sizes)

Image Sizes for Each Attack Type:
{'Normal': (9, 7, 3), 'DDoS_UDP': (9, 7, 3), 'DDoS_ICMP': (9, 7, 3), 'DDoS_TCP': (9, 7, 3), 'Password': (9, 7, 3), 'DDoS_HTTP': (9, 7, 3), 'Vulnerability_scanner': (9, 7, 3), 'SQL_injection': (9, 7, 3), 'Backdoor': (9, 7, 3), 'Uploading': (9, 7, 3), 'Port_Scanning': (9, 7, 3), 'XSS': (9, 7, 3), 'Ransomware': (9, 7, 3), 'MITM': (9, 7, 3), 'OS_Fingerprinting': (9, 7, 3)}


In [10]:
df.isna().sum().to_numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
df.describe()

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,http.file_data,http.content_length,...,tcp.seq,tcp.srcport,udp.stream,dns.qry.name,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.ver,mbtcp.len,Attack_label
count,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,...,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07,1.979325e+07
mean,3.995409e-01,8.351467e-02,7.337561e-02,9.174593e-03,4.666129e-03,3.467085e-03,7.163670e-02,7.312306e-02,2.518825e-03,3.211657e-03,...,1.406969e-02,1.662181e-01,7.584121e-02,6.035645e-03,6.197676e-02,1.898157e-02,7.511431e-02,3.530087e-03,2.055706e-03,4.754298e-03
std,2.510678e-01,1.671260e-01,1.545721e-01,5.267590e-02,2.541018e-02,2.140060e-02,2.035858e-01,2.016080e-01,4.399235e-02,4.179979e-02,...,8.332682e-02,2.634009e-01,2.071010e-01,5.816266e-02,1.821060e-01,7.211452e-02,2.251489e-01,3.210668e-02,3.561558e-02,6.878732e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.839602e-01,1.418118e-05,1.360230e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,1.527184e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.803702e-01,1.890824e-05,2.720459e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.360528e-05,3.833232e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.975213e-01,6.638682e-02,8.093366e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.488423e-04,1.655925e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [5]:
pip install torch==1.9.1

ERROR: Could not find a version that satisfies the requirement torch==1.9.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.9.1


In [6]:
pip -q install git+https://github.com/alok-ai-lab/pyDeepInsight.git#egg=pyDeepInsight

  Preparing metadata (setup.py) ... done


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyDeepInsight import ImageTransformer

# Replace 'your_data.csv' with the filename of your dataset
# Assuming you already have your data in a DataFrame named 'df'
# If you have not loaded your data into 'df', replace this with the appropriate code

# Extract features and labels
X = df.drop(columns=['Attack_type']).values
y = df['Attack_type'].values

# Define the class distribution
class_distribution = {
    "Normal": 10066373,
    "DDoS_UDP": 3201626,
    "DDoS_ICMP": 2914354,
    "DDoS_TCP": 2020120,
    "Password": 1053385,
    "DDoS_HTTP": 229022,
    "Vulnerability_scanner": 145869,
    "SQL_injection": 51203,
    "Backdoor": 49724,
    "Uploading": 37634,
    "Port_Scanning": 22564,
    "XSS": 15915,
    "Ransomware": 10925,
    "MITM": 2458,
    "OS_Fingerprinting": 2002
}

# Initialize the ImageTransformer
transformer = ImageTransformer(feature_extractor="pca", pixels=50)

# Output directory path
output_dir = '/content/drive/MyDrive/Images/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Calculate the number of samples and batch size
num_samples = X.shape[0]
batch_size = 50  # You can adjust this batch size based on your memory capacity

# Process data in batches
for i in range(0, num_samples, batch_size):
    start_idx = i
    end_idx = min(i + batch_size, num_samples)
    X_batch = X[start_idx:end_idx]
    y_batch = y[start_idx:end_idx]

    # Fit the transformer on the current batch
    transformer.fit(X_batch, y_batch)

    # DeepInsight transformation for the current batch
    X_transformed = transformer.transform(X_batch)

    # Save the images and labels for the current batch
    for j, image in enumerate(X_transformed):
        # Generate a unique filename for each image
        image_filename = os.path.join(output_dir, f'image_{start_idx + j}_label_{y_batch[j]}.png')
        plt.imsave(image_filename, image, cmap='viridis')

    print(f"Batch {i//batch_size + 1} processed.")

print("All batches processed successfully.")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Batch 29768 processed.
Batch 29769 processed.
Batch 29770 processed.
Batch 29771 processed.
Batch 29772 processed.
Batch 29773 processed.
Batch 29774 processed.
Batch 29775 processed.
Batch 29776 processed.
Batch 29777 processed.
Batch 29778 processed.
Batch 29779 processed.
Batch 29780 processed.
Batch 29781 processed.
Batch 29782 processed.
Batch 29783 processed.
Batch 29784 processed.
Batch 29785 processed.
Batch 29786 processed.
Batch 29787 processed.
Batch 29788 processed.
Batch 29789 processed.
Batch 29790 processed.
Batch 29791 processed.
Batch 29792 processed.
Batch 29793 processed.
Batch 29794 processed.
Batch 29795 processed.
Batch 29796 processed.
Batch 29797 processed.
Batch 29798 processed.
Batch 29799 processed.
Batch 29800 processed.
Batch 29801 processed.
Batch 29802 processed.
Batch 29803 processed.
Batch 29804 processed.
Batch 29805 processed.
Batch 29806 processed.
Batch 29807 processed.
Ba